# Purpose

The purpose of this function is to redo the function that adds the invoicing data and calculates the sum of squares. There are 2 reasons for this:
1. That function is a mess. The intention is that this function will be much cleaner
2. We want to include the time series for the ICIO for the own consumption shares. I will do that in this function.


### Let's begin
1. First let's load in packages and set paths for loading in data

In [1]:
# packages
import pandas as pd
import numpy as np
import pickle
from ipywidgets import FloatProgress
import matplotlib.pyplot as plt
import statsmodels.api as sm
import re
import statsmodels.tsa.x13 as X13
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.dates as pltdates
import matplotlib.patches as mpatches
import matplotlib as matplotlib
from IPython.display import display
import argparse
import os
import subprocess
from datetime import datetime
import scipy as scipy
import xlrd as xlrd
import country_converter as coco
import datetime as dt
from dotenv import dotenv_values, find_dotenv
import seaborn as sns
import pyreadr
# this looks for your configuration file and then reads it as a dictionary
config = dotenv_values(find_dotenv())

# Set parameters and paths
path_cleandata = config["CLEANDATA"] + '/'
path_rawdata = os.path.abspath(config["RAWDATA"]) + '/'
path_ICIO = os.path.abspath(config["RAWDATA"]) + '/ICIO/'
path_invoicing = os.path.abspath(config["RAWDATA"]) + '/'

# options
extrapolate = "simple" # Set to "simple" or "slope" to extrapolate forward and backwards based on the level (simple) or by extimating a slope and using that (slope option). Note slope only currently goes backwards (why Nick?)
extrapolateforward = True # set to true to extrapolate forward
saveAppend = '_ICIO_Nov14_2023_noConsumption'
# Options on constructing consumption currency shares and production currency shares
FullHomeShare = 1  # Set to 1 to assume 100% of home consumption is in home currency
NoTradableICIOAdj = 0 # Set to 0 to take the whole ICIO and not adjust for which sectors are tradable/non-tradable. Set to 1 to adjust for tradable based on a manual classification. Set to 2 to adjust based on dropping all shares above 0.9
ownConsumptionThreshold = 0.9 # if NoTradableICIOAdj=2, then this sets the threshold by which a sector is tradable vs. non-tradable

skipConsumptionAdjustment = 1 # set to 1 to skip the consumption adjustments of the import/export shares

2. Load in the portfolio and currency data. 

In [2]:
portfoliodata = pd.read_pickle(path_cleandata+'PortfolioData_interestrates.pkl')
# get time series of currency shares imports_currency_shares_time_series.xlsx

# dataframe that will store the actually available data
currency_TS = pd.read_excel(open(path_invoicing+'imports_currency_shares_time_series.xlsx','rb'),sheet_name='Annual')

# Full sample is 1970 to 2023 so we need to add some additional rows so that currency_TS covers this sample when interpolating
years = range(1970,2024)
countries = currency_TS['Country'].unique()
Country = [c for c in countries for y in years]
year = [y for c in countries for y in years]

tomerge = pd.DataFrame(data=Country)
tomerge=tomerge.rename(columns={0:'Country'})
tomerge['year']=year



currency_TS = pd.merge(currency_TS,tomerge,on=['Country','year'],how='outer')


# Fill in necessary Country names iso3 codes and imf codes
currency_TS['ISO3C'] = currency_TS.groupby('Country')['ISO3C'].transform('first')
currency_TS['imfcode'] = currency_TS.groupby('Country')['imfcode'].transform('first')
currency_TS = currency_TS[currency_TS['ISO3C'].notnull()]

# Define a new data frame to store interpolated currency values so we can play around with interpolation or not.
currency_TS_I = currency_TS.copy()
currency_TS_I = currency_TS_I.rename(columns={c: c+'_I' for c in currency_TS_I.columns if c not in ['Country', 'ISO3C', 'imfcode', 'year']})



3. Interpolation and extrapolation is done in this block

In [3]:
# Function that extrapolates based on the last oberved value or backwards with first observed value
def first_last_nams(df,var,cnty):
    f_var = var+'_first'
    f_var_d = var+'_first_date'
        
    df.loc[:,f_var_d]=df[df[var].notnull()].groupby(cnty)['year'].transform('first')
    l_var = var+'_last'
    l_var_d = var+'_last_date'
    df.loc[:,l_var_d]=df[df[var].notnull()].groupby(cnty)['year'].transform('last')
    df.loc[:,f_var_d] = df.groupby(cnty)[f_var_d].transform('first')
    df.loc[:,l_var_d] = df.groupby(cnty)[l_var_d].transform('last')
    
    df.loc[:,f_var] = df.groupby(cnty)[var].transform('first')
    df.loc[:,l_var] = df.groupby(cnty)[var].transform('last')
    
    df.loc[df['year']<df[f_var_d],var]=df[f_var][df['year']<df[f_var_d]]
    df.loc[df['year']>df[l_var_d],var]=df[l_var][df['year']>df[l_var_d]]
    
    df = df.drop(columns = [l_var,l_var_d,f_var,f_var_d])
    
    return df

# Interpolate (always do this)
currency_TS_I = currency_TS_I.groupby('ISO3C').apply(lambda group: group.interpolate(limit_area='inside'))

# extrapolate simple
if extrapolate=="simple":
    currency_TS_I = first_last_nams(currency_TS_I,'Export USD_I','Country')
    currency_TS_I = first_last_nams(currency_TS_I,'Export EUR_I','Country')
    currency_TS_I = first_last_nams(currency_TS_I,'Export Home_I','Country')
    currency_TS_I = first_last_nams(currency_TS_I,'Export Other_I','Country')
    currency_TS_I = first_last_nams(currency_TS_I,'Export Unclassified_I','Country')
    currency_TS_I = first_last_nams(currency_TS_I,'Export Other_Excl_USDEUR_I','Country')
    currency_TS_I = first_last_nams(currency_TS_I,'Import USD_I','Country')
    currency_TS_I = first_last_nams(currency_TS_I,'Import EUR_I','Country')
    currency_TS_I = first_last_nams(currency_TS_I,'Import Home_I','Country')
    currency_TS_I = first_last_nams(currency_TS_I,'Import Other_I','Country')
    currency_TS_I = first_last_nams(currency_TS_I,'Import Unclassified_I','Country')
    currency_TS_I = first_last_nams(currency_TS_I,'Import Other_Excl_USDEUR_I','Country')
elif extrapolate=="slope":
    extrapolateVarList = ['Export Home','Import Home','Export USD','Import USD','Export EUR',\
                      'Import EUR','Export Other','Import Other']
    clist  = currency_TS['Country'].unique()
    for c in clist:
        for v in extrapolateVarList:
            # regress
            temp = currency_TS[currency_TS['Country']==c][[v,'year']].dropna()
            if len(temp)>2:
                mod = sm.OLS(temp[v],sm.add_constant(temp['year'])).fit()

                # Backwards extrapolate
                firstYear = temp['year'].min()
                currency_TS_I.loc[(currency_TS_I['Country']==c)&(currency_TS_I['year']<firstYear),v+'_I'] = mod.params[0]+mod.params[1]*currency_TS_I.loc[(currency_TS_I['Country']==c)&(currency_TS_I['year']<firstYear),'year']
                if extrapolateforward:
                    # forwards extrapolate
                    lastYear = temp['year'].max()
                    currency_TS_I.loc[(currency_TS_I['Country']==c)&(currency_TS_I['year']>lastYear),v+'_I'] = mod.params[0]+mod.params[1]*currency_TS_I.loc[(currency_TS_I['Country']==c)&(currency_TS_I['year']>lastYear),'year']

                # Make sure that we never get a currency share above 0 or below 100.
                currency_TS_I.loc[(currency_TS_I['Country']==c)&(currency_TS_I[v+'_I']>100),v+'_I']=100
                currency_TS_I.loc[(currency_TS_I['Country']==c)&(currency_TS_I[v+'_I']<0),v+'_I']=0
                currency_TS_I.loc[(currency_TS_I['Country']==c)&(currency_TS_I[v+'_I']>100),v+'_I']=100
                currency_TS_I.loc[(currency_TS_I['Country']==c)&(currency_TS_I[v+'_I']<0),v+'_I']=0

4. Merge the original currency share data (with nans for misisng) with the interpolated/extrapolated data. Then do some additional cleaning (adding 0s when one value is nan but another in the same country x month is non-nan)

In [4]:
currency_TS_all = pd.merge(left=currency_TS, right=currency_TS_I, how='inner', on=['Country', 'ISO3C', 'imfcode', 'year'])

notnullInd = currency_TS_all['Export USD'].notnull()|currency_TS_all['Export EUR'].notnull()|currency_TS_all['Export Home'].notnull()|currency_TS_all['Export Other'].notnull()

# put zeros in rows where one currency share is not empty
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Export USD'].isnull()),'Export USD']=0
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Export EUR'].isnull()),'Export EUR']=0
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Export Home'].isnull()),'Export Home']=0
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Export Other'].isnull()),'Export Other']=0

notnullInd = currency_TS_all['Export USD_I'].notnull()|currency_TS_all['Export EUR_I'].notnull()|currency_TS_all['Export Home_I'].notnull()|currency_TS_all['Export Other_I'].notnull()

# put zeros in rows where one currency share is not empty
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Export USD_I'].isnull()),'Export USD_I']=0
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Export EUR_I'].isnull()),'Export EUR_I']=0
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Export Home_I'].isnull()),'Export Home_I']=0
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Export Other_I'].isnull()),'Export Other_I']=0

notnullInd = currency_TS_all['Import USD'].notnull()|currency_TS_all['Import EUR'].notnull()|currency_TS_all['Import Home'].notnull()|currency_TS_all['Import Other'].notnull()

# put zeros in rows where one currency share is not empty
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Import USD'].isnull()),'Import USD']=0
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Import EUR'].isnull()),'Import EUR']=0
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Import Home'].isnull()),'Import Home']=0
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Import Other'].isnull()),'Import Other']=0

notnullInd = currency_TS_all['Import USD_I'].notnull()|currency_TS_all['Import EUR_I'].notnull()|currency_TS_all['Import Home_I'].notnull()|currency_TS_all['Import Other_I'].notnull()

# put zeros in rows where one currency share is not empty
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Import USD_I'].isnull()),'Import USD_I']=0
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Import EUR_I'].isnull()),'Import EUR_I']=0
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Import Home_I'].isnull()),'Import Home_I']=0
currency_TS_all.loc[(notnullInd==True)&(currency_TS_all['Import Other_I'].isnull()),'Import Other_I']=0


5. Load in ICIO data, focus on tradable sectors and calculate own consumption shares. 

In [5]:
icio_shares = pd.DataFrame(columns=['year','iso3','share','share_export'])
Sector_names = pd.read_excel(path_ICIO+'ReadMe_ICIO_R.xlsx',sheet_name='Country_Industry',usecols='I,J', header=2)  
Sector_names = Sector_names.dropna()
# Based on the above data here is my manual tradable classification
ManualTradableInd = [0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,26,27,28,30,32,34,35,37]
sectorKeep = Sector_names.loc[ManualTradableInd]
for year in range(1995,2021,1):
    print(year)
    # get data from year
    data = pyreadr.read_r(path_ICIO + str(year) + '.rdata')['temp.icio']
    data=data.reset_index().rename(columns={'index': 'sector_code'})
    data['iso3'] = data['sector_code'].str.split('_').str[0]
    data.loc[(data['iso3']=='MX1')|(data['iso3']=='MX2'),'iso3']='MEX'
    data.loc[(data['iso3']=='CN1')|(data['iso3']=='CN2'),'iso3']='CHN'
    data['sector'] = data['sector_code'].str.slice(start=4)
    export = data.copy()
    # get rid of columns and rows that i dont need
    data=data.set_index('sector_code')
    data = data.loc[:, data.columns.str.endswith("HFCE")].reset_index()
    data.drop(data[data['sector_code'].str.endswith(('TLS', 'VA', 'OUT'))].index, inplace=True)
    data.drop(data[data['sector_code'].str.startswith(('MX1', 'MX2', 'CN1', 'CN2'))].index, inplace=True)
    data['iso3'] = data['sector_code'].str.split('_').str[0]
    data.loc[(data['iso3']=='MX1')|(data['iso3']=='MX2'),'iso3']='MEX'
    data.loc[(data['iso3']=='CN1')|(data['iso3']=='CN2'),'iso3']='CHN'
    data['sector'] = data['sector_code'].str.slice(start=4)
    
    # do the export shares - these are exports/ C+I+G+NX - first I need to calculate exports and then C+I+G+NX
    #export = export[['sector_code','iso3','ARG_A01_02','ARG_A03','AUS_A01_02','AUS_A03','ARG_HFCE','AUS_HFCE']] . # For testing - keep commented out when running correctly
    export['Exports']=0
    export['CIGX']=0
    
    export_cols = export.columns
    export.drop(export[export['sector_code'].str.endswith(('TLS','VA','OUT'))].index, inplace=True)
    for c in export['iso3'].unique():
        export.loc[export['iso3']==c,'Exports']=export.loc[export['iso3']==c,(export_cols.str.startswith(c)==False)&(export_cols!='Exports')&(export_cols!='CIGX')].sum(numeric_only=True,axis=1)
    export['CIGX']=export.loc[:,(export_cols!='Exports')&(export_cols!='CIGX')].sum(numeric_only=True,axis=1) # need to make sure exports column (non-zero now) isn't included in sum


    # Remove sectors we deem to be non-tradable
    if NoTradableICIOAdj==1:
        # criteria to remove is Nick's manual classification
        data=data[data['sector'].isin(sectorKeep['Code.1'])]
        export=export[export['sector'].isin(sectorKeep['Code.1'])]
    elif NoTradableICIOAdj==2:
        # Criteria to remove is based on the own consumption share being larger than some threshold
        data_sumCheck=data.groupby('sector').sum(numeric_only=True).reset_index()
        data_2 = pd.merge(data,data_sumCheck,on='sector',how='outer')
        FD_list =data.columns
        FD_list = FD_list[data.columns.str.endswith('HFCE')]
        for li in FD_list:
            data_2[li+'_x']=data_2[li+'_x']/data_2[li+'_y']
        shares_df = pd.DataFrame({'sector':data['sector'].unique()})
        shares_df = shares_df.set_index('sector')
        for c in data['iso3'].unique():
            col = c+'_HFCE_x'
            temp = data_2[[col,'sector','iso3']][data_2['iso3']==c]
            temp[col] = temp[col]<ownConsumptionThreshold
            temp=temp.set_index('sector')
            shares_df[col]=temp[col]
        data = pd.merge(data,shares_df,on='sector',how='outer')
        export = pd.merge(export,shares_df,on='sector',how='outer')
        for c in data['iso3'].unique():
            col = c+'_HFCE'
            col2 = c+'_HFCE_x'
            data[col] = data[col]*data[col2]
            export.loc[export['iso3']==c,'Exports'] = export.loc[export['iso3']==c,"Exports"]*export.loc[export['iso3']==c,col2]  # for country c scale Exports and CIGX by the relevant col2 T/F indicator for tradable
            export.loc[export['iso3']==c,'CIGX'] = export.loc[export['iso3']==c,"CIGX"]*export.loc[export['iso3']==c,col2]
        tokeep = data.columns
        keepInd = data.columns.isin(data['iso3'].unique()+'_HFCE')
        keepInd[data.columns=='iso3']=True
        tokeep = tokeep[keepInd]
        data = data[tokeep]
        export = export[['iso3','Exports','CIGX']]
    # take sum over industries by country
    data_reshape = data.groupby(['iso3']).sum(numeric_only=True).reset_index()
    export= export.groupby('iso3').sum(numeric_only=True)
    data_reshape = pd.merge(data_reshape,export.reset_index()[['iso3','Exports','CIGX']],on='iso3',how='outer')
    # display(data_reshape.head())

    # go over each row (kinda slow?) and get the values i need
    for index, row in data_reshape.iterrows():
        iso3 = row['iso3']
        # print(iso3)i
        value = row[iso3 + '_HFCE']
        # print(value)
        sum_iso3 = sum(data_reshape[iso3 + '_HFCE'])
        share = value/sum_iso3
        share_export = row['Exports']/row['CIGX']
        # print(share)

        # append a new row to the new df
        icio_shares.loc[len(icio_shares)] = [year, iso3, share,share_export]

1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


6. Merge this with the other currency share data

In [6]:
#portfoliodata = portfoliodata[portfoliodata['Date'].notnull()]
icio_shares['year'] = icio_shares['year'].astype('Int64')
currency_TS_all['year'] = currency_TS_all['year'].astype('Int64')
#portfoliodata = pd.merge(portfoliodata,icio_shares,on=['year','iso3'],how='left')
currency_TS_all=currency_TS_all.rename(columns={"ISO3C":"iso3"})
df_all = pd.merge(icio_shares,currency_TS_all,on=['year','iso3'],how='outer',indicator=True)
df_all = df_all.sort_values(by=['iso3','year'])
df_all = first_last_nams(df_all,'share','iso3')
df_all = first_last_nams(df_all,'share_export','iso3')

7. Create a Euro Indicator for the Euro Area countries

In [7]:
Euro_list = ['Austria','Belgium','Cyprus','Germany','Estonia',
            'Spain','Finland','France','Greece','Ireland','Italy',
            'Lithuania','Luxembourg','Latvia','Monaco',
            'Malta','Netherlands','Portugal','Slovenia','Slovakia',
            'Vatican City']
df_all['Euro Indicator']=df_all['Country'].isin(Euro_list)

8. Adjust the shares from Import/Export shares to Consumption/CIGX shares

In [8]:
importcols = ['Import USD','Import EUR', 'Import Home', 'Import Other', 'Import Unclassified','Import Other_Excl_USDEUR', 
              'Import USD_I','Import EUR_I', 'Import Home_I', 'Import Other_I', 'Import Unclassified_I','Import Other_Excl_USDEUR_I']
exportcols = ['Export USD','Export EUR', 'Export Home', 'Export Other', 'Export Unclassified','Export Other_Excl_USDEUR',
              'Export USD_I','Export EUR_I', 'Export Home_I', 'Export Other_I', 'Export Unclassified_I','Export Other_Excl_USDEUR_I']



# For countries with empty or 0 entries in import home or export home, do the sum as 1- import Euro -Import US - Import Other
df_all['index'] = False
df_all.loc[((df_all['Import Home']==0)|df_all['Import Home'].isnull())&\
           (df_all['Import USD']+df_all['Import EUR']+df_all['Import Other']<100),'index']=True
df_all.loc[df_all['index']==True,'Import Home']=100-(df_all['Import USD']+df_all['Import EUR']+df_all['Import Other'])


df_all['index'] = False
df_all.loc[((df_all['Export Home']==0)|df_all['Export Home'].isnull())&\
           (df_all['Export USD']+df_all['Export EUR']+df_all['Export Other']<100),'index']=True
df_all.loc[df_all['index']==True,'Export Home']=100-(df_all['Export USD']+df_all['Export EUR']+df_all['Export Other'])

df_all['index']=True

# apply the import home currency adjustment
df_all['share'] = df_all['share']*FullHomeShare # scale the share of own consumption by the share of own consumption in home currency (setting above). This assumes
                                                # that the remaining parts of own consumption are split the same as the import data
df_all['share_export'] = 1-((1-df_all['share_export'])*FullHomeShare) # Scale the share of exports in a consistent manner as above



# make sure import consumption shares are not above 1
df_all.loc[df_all['share']>1,'share']=1
df_all.loc[df_all['share_export']>1,'share_export']=1

# Now scale all the import shares by own tradable shares
if skipConsumptionAdjustment:
    for imp in importcols:
        newnam = imp+'_old'
        df_all.loc[:,newnam] = df_all[imp] # save the original value
        # Update the import shares to be consumption shares
        df_all.loc[df_all['index'],imp] = df_all.loc[df_all['index'],imp]*(1-df_all.loc[df_all['index'],'share'])
        if (imp=='Import Home')|(imp=='Import Home_I'):
            df_all.loc[df_all['index'],imp] = df_all.loc[df_all['index'],imp] + 100*df_all.loc[df_all['index'],'share']
            
            
    for exp in exportcols:
        newnam = exp+'_old'
        df_all.loc[:,newnam] = df_all[exp] # save old value

        # update export shares to be CIGX shares (though Nick: should we make this GDP shares?!?)
        df_all.loc[df_all['index'],exp] = df_all.loc[df_all['index'],exp]*(df_all.loc[df_all['index'],'share_export'])
        if (exp == 'Export Home')|(exp == 'Export Home_I'):
                df_all.loc[df_all['index'],exp] = df_all.loc[df_all['index'],exp] + 100*(1-df_all.loc[df_all['index'],'share_export'])
    
# Add the Euro column to the home column if the country is in the Eurozone (and for Denamrk which pegs to the Euro)
df_all.loc[(df_all['Country']=='Denmark')&(df_all['year']>=1999),'Euro Indicator']=True # Denmark Pegged to Euro
df_all.loc[(df_all['Country']=='Switzerland')&(df_all['year'].isin([2011,2012,2013,2014,2015])),'Euro Indicator']=True # Swizz franc temporarily Pegged to Euro

df_all.loc[df_all['Euro Indicator']==1,'Import EUR'] = df_all.loc[df_all['Euro Indicator']==1,'Import EUR']+df_all.loc[df_all['Euro Indicator']==1,'Import Home']
df_all.loc[df_all['Euro Indicator']==1,'Import EUR_I'] = df_all.loc[df_all['Euro Indicator']==1,'Import EUR_I']+df_all.loc[df_all['Euro Indicator']==1,'Import Home_I']
df_all.loc[df_all['Euro Indicator']==1,'Export EUR'] = df_all.loc[df_all['Euro Indicator']==1,'Export EUR']+df_all.loc[df_all['Euro Indicator']==1,'Export Home']
df_all.loc[df_all['Euro Indicator']==1,'Export EUR_I'] = df_all.loc[df_all['Euro Indicator']==1,'Export EUR_I']+df_all.loc[df_all['Euro Indicator']==1,'Export Home_I']
df_all.loc[df_all['Euro Indicator']==1,'Import Home'] = 0
df_all.loc[df_all['Euro Indicator']==1,'Import Home_I'] = 0
df_all.loc[df_all['Euro Indicator']==1,'Export Home'] = 0
df_all.loc[df_all['Euro Indicator']==1,'Export Home_I'] = 0

# turn off Denmark for Aggregation reasons of import share for Euro
df_all.loc[(df_all['Country']=='Denmark')&(df_all['year']>=1999),'Euro Indicator']=False # Denmark Pegged to Euro
df_all.loc[(df_all['Country']=='Switzerland')&(df_all['year'].isin([2011,2012,2013,2014,2015])),'Euro Indicator']=False # Swizz franc temporarily Pegged to Euro

# Add the USD column to the home column if the country is the US
df_all.loc[df_all['Country']=='United States','Import USD'] = df_all.loc[df_all['Country']=='United States','Import USD']+df_all.loc[df_all['Country']=='United States','Import Home']
df_all.loc[df_all['Country']=='United States','Import USD_I'] = df_all.loc[df_all['Country']=='United States','Import USD_I']+df_all.loc[df_all['Country']=='United States','Import Home_I']
df_all.loc[df_all['Country']=='United States','Export USD'] = df_all.loc[df_all['Country']=='United States','Export USD']+df_all.loc[df_all['Country']=='United States','Export Home']
df_all.loc[df_all['Country']=='United States','Export USD_I'] = df_all.loc[df_all['Country']=='United States','Export USD_I']+df_all.loc[df_all['Country']=='United States','Export Home_I']
df_all.loc[df_all['Country']=='United States','Import Home'] = 0
df_all.loc[df_all['Country']=='United States','Import Home_I'] = 0
df_all.loc[df_all['Country']=='United States','Export Home'] = 0
df_all.loc[df_all['Country']=='United States','Export Home_I'] = 0

# drop _old columns
droplist = df_all.columns
droplist_I = [item for i,item in enumerate(droplist) if '_old' in item or 'Unclassified' in item or 'USDEUR' in item or '_I' not in item]

droplist = [item for i,item in enumerate(droplist) if '_old' in item or 'Unclassified' in item or 'USDEUR' in item or '_I' in item]


In [9]:
df_all[df_all['Country']=='Estonia'][['Country','year','index','Import USD_I','Import EUR_I','Import Home_I','Import Other_I','Import USD','Import EUR','Import Home','Import Other','Import USD_old','Import EUR_old','Import Home_old','Import Other_old','share','share_export']]





,Country,year,index,Import USD_I,Import EUR_I,Import Home_I,Import Other_I,Import USD,Import EUR,Import Home,Import Other,Import USD_old,Import EUR_old,Import Home_old,Import Other_old,share,share_export
3961,Estonia,1970,True,5.181380,89.788155,0.0,5.030465,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.748477,0.621272
3962,Estonia,1971,True,5.181380,89.788155,0.0,5.030465,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.748477,0.621272
3963,Estonia,1972,True,5.181380,89.788155,0.0,5.030465,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.748477,0.621272
3964,Estonia,1973,True,5.181380,89.788155,0.0,5.030465,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.748477,0.621272
3965,Estonia,1974,True,5.181380,89.788155,0.0,5.030465,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.748477,0.621272
3966,Estonia,1975,True,5.181380,89.788155,0.0,5.030465,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.748477,0.621272
3967,Estonia,1976,True,5.181380,89.788155,0.0,5.030465,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.748477,0.621272
3968,Estonia,1977,True,5.181380,89.788155,0.0,5.030465,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.748477,0.621272
3969,Estonia,1978,True,5.181380,89.788155,0.0,5.030465,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.748477,0.621272
3970,Estonia,1979,True,5.181380,89.788155,0.0,5.030465,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.748477,0.621272


In [10]:
# Calculate the relevant sum of shares variables (2 types: one ignoring other and one including it as a single currency)
cols = df_all.columns
importcols = cols[[i for i,item in enumerate(cols) if 'Import' in item and item not in droplist]]
exportcols = cols[[i for i,item in enumerate(cols) if 'Export' in item  and item not in droplist]]

df_all['OtherCurrencySum']=df_all[importcols].sum(axis=1)
nanInd = df_all[importcols].isnull().sum(axis=1)
df_all.loc[nanInd,'OtherCurrencySum']=np.nan

df_all['remainder'] = 100-df_all['OtherCurrencySum']
for imp in importcols:
    df_all.loc[df_all['remainder']<-10**(-1),imp]=    df_all.loc[df_all['remainder']<-10**(-1),imp]/df_all.loc[df_all['remainder']<-10**(-1),'OtherCurrencySum']
    df_all.loc[df_all['remainder']>10**(-5),imp]=    df_all.loc[df_all['remainder']>10**(-5),imp]*100/(df_all.loc[df_all['remainder']>-10**(-5),'OtherCurrencySum'])
#df_all['OtherCurrencySum']=pd.DataFrame(np.asmatrix(df_all[importcols]).T).sum()
#df_all['remainder'] = 100-df_all['OtherCurrencySum']    
    
# Now sum the squared import shares
df_all['SumShares']=0
for imp in importcols:
    df_all.loc[df_all[imp].notnull(),'SumShares']+=(df_all.loc[df_all[imp].notnull(),imp]/100)**2
df_all.loc[df_all['SumShares']==0,'SumShares']=np.nan    
nam = 'SumShares' 


# export shares
df_all['OtherCurrencySum_X']=df_all[importcols].sum(axis=1)
df_all['remainder_X'] = 100-df_all['OtherCurrencySum_X']
for exp in exportcols:
    df_all.loc[df_all['remainder_X']<-10**(-1),exp]=    df_all.loc[df_all['remainder_X']<-10**(-1),exp]/df_all.loc[df_all['remainder_X']<-10**(-1),'OtherCurrencySum_X']
    df_all.loc[df_all['remainder_X']>10**(-5),exp]=    df_all.loc[df_all['remainder_X']>-10**(-5),exp]*100/(df_all.loc[df_all['remainder_X']>-10**(-5),'OtherCurrencySum_X'])
    
df_all['OtherCurrencySum_X']=pd.DataFrame(np.asmatrix(df_all[exportcols]).T).sum()
df_all['remainder_X'] = 100-df_all['OtherCurrencySum_X']    
    
# Now sum the squared eport shares
df_all['SumShares_X']=0
for exp in exportcols:
    df_all.loc[df_all[exp].notnull(),'SumShares_X']+=(df_all.loc[df_all[exp].notnull(),exp]/100)**2
df_all.loc[df_all['SumShares_X']==0,'SumShares_X']=np.nan

# add in 0s for Norway
#df_all.loc[df_all['SumShares_X'].isnull(),'SumShares_X']=0

importcols = cols[[i for i,item in enumerate(cols) if 'Import' in item and item not in droplist_I]]
exportcols = cols[[i for i,item in enumerate(cols) if 'Export' in item  and item not in droplist_I]]

df_all['OtherCurrencySum']=df_all[importcols].sum(axis=1)
df_all['remainder'] = 100-df_all['OtherCurrencySum']
for imp in importcols:
    df_all.loc[df_all['remainder']<-10**(-1),imp]=    df_all.loc[df_all['remainder']<-10**(-1),imp]/df_all.loc[df_all['remainder']<-10**(-1),'OtherCurrencySum']
    df_all.loc[df_all['remainder']>10**(-5),imp]=    df_all.loc[df_all['remainder']>-10**(-5),imp]*100/(df_all.loc[df_all['remainder']>-10**(-5),'OtherCurrencySum'])
df_all['OtherCurrencySum']=pd.DataFrame(np.asmatrix(df_all[importcols]).T).sum()
df_all['remainder'] = 100-df_all['OtherCurrencySum']    
    
# Now sum the squared import shares
df_all['SumShares_I']=0
for imp in importcols:
    df_all.loc[df_all[imp].notnull(),'SumShares_I']+=(df_all.loc[df_all[imp].notnull(),imp]/100)**2
df_all.loc[df_all['SumShares_I']==0,'SumShares_I']=np.nan    
nam = 'SumShares_I' 

# export shares
df_all['OtherCurrencySum_X']=pd.DataFrame(np.asmatrix(df_all[exportcols]).T).sum()
df_all['remainder_X'] = 100-df_all['OtherCurrencySum_X']
for exp in exportcols:
    df_all.loc[df_all['remainder_X']<-10**(-1),exp]=    df_all.loc[df_all['remainder_X']<-10**(-1),exp]/df_all.loc[df_all['remainder_X']<-10**(-1),'OtherCurrencySum_X']
    df_all.loc[df_all['remainder_X']>10**(-5),exp]=    df_all.loc[df_all['remainder_X']>-10**(-5),exp]*100/(df_all.loc[df_all['remainder_X']>-10**(-5),'OtherCurrencySum_X'])
    
df_all['OtherCurrencySum_X']=pd.DataFrame(np.asmatrix(df_all[exportcols]).T).sum()
df_all['remainder_X'] = 100-df_all['OtherCurrencySum_X']    
    
# Now sum the squared eport shares
df_all['SumShares_X_I']=0
for exp in exportcols:
    df_all.loc[df_all[exp].notnull(),'SumShares_X_I']+=(df_all.loc[df_all[exp].notnull(),exp]/100)**2
df_all.loc[df_all['SumShares_X_I']==0,'SumShares_X_I']=np.nan

# add in 0s for Norway
#df_all.loc[df_all['SumShares_X_I'].isnull(),'SumShares_X_I']=0

In [11]:
# merge with portfolio data
df_all.rename(columns={"ISO3C": "iso3"}, inplace=True)
portfoliodata['year'] = portfoliodata['year'].astype(float)

# [NICK] Add in a month variable and delete any entries where month does not exist
portfoliodata['month'] = portfoliodata['Date'].dt.month
portfoliodata = portfoliodata[portfoliodata['month'].notnull()]

# truncate columns of df_all for Julien to have a parsimonious dataseries
# df_all_par = df_all[['Country','year','ISO2','iso3','imfcode','Import USD_I','Import EUR_I','Import Home_I','Import USD','Import EUR','Import Home','SumShares','SumShares_I','SumShares_X','SumShares_X_I']]
df_all_par = df_all[['Country','year','iso3','imfcode',
                     'Import USD_I','Import EUR_I','Import Home_I','Import USD','Import EUR','Import Home', 'Import Other', 'Import Other_I',
                     'Export USD_I','Export EUR_I','Export Home_I','Export USD','Export EUR','Export Home', 'Export Other', 'Export Other_I',
                     'SumShares','SumShares_I','SumShares_X','SumShares_X_I','Import USD_old','Import EUR_old']]

# merge with other data
df_all_par.loc[:,'year'] = df_all_par.loc[:,'year'].astype('Float64')
df_all_par = pd.merge(left=df_all_par, right=portfoliodata, how='outer', on=['iso3','year'])
df_all_par.sort_values(by=['iso3', 'year'], inplace=True)

/var/folders/hx/h9ft_v812fs68lgk203tqsq80000gn/T/ipykernel_66768/4266358202.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_par.loc[:,'year'] = df_all_par.loc[:,'year'].astype('Float64')


In [12]:
# fill in year and month after merge
df_all_par['year'] = df_all_par['year'].fillna(df_all_par['Date'].dt.year.astype('Int64'))
df_all_par['month'] = df_all_par['month'].fillna(df_all_par['Date'].dt.month.astype('Int64'))
df_all=df_all_par

In [13]:
# Create a DATE BASED Euro Indicator
cc = coco.CountryConverter()
euro = pd.read_csv(path_rawdata+'Euro_Yield_Dates.csv', encoding='latin-1')
euro['EntryDate'] = pd.DatetimeIndex(pd.to_datetime(euro['Date'], format='%d/%m/%Y')).year
euro['Date'] = None
euro['iso2'] = euro['Code'].str[:2]
euro['iso3'] = cc.pandas_convert(series=euro['iso2'], to='ISO3')
euro = euro.drop(columns=['Currency','Code','Date','iso2'])

df_all = pd.merge(df_all,euro,on='iso3',how='left')

df_all['Euro Indicator']=False
df_all.loc[df_all['year']>=df_all['EntryDate'],'Euro Indicator']=True

# final thing: find currency shares for the Euro Area:
wdiin = pd.read_pickle(path_cleandata+'WDI.pkl')

wdiin = wdiin[wdiin['gdp_cur_dol'].notnull()][['iso3','year','gdp_cur_dol']]
df_all['year']=df_all['year'].astype('int')

df_all = pd.merge(left=df_all,right=wdiin,on=['iso3','year'],how='left')


# Eu GDP shares
df_all.loc[df_all['year']>2019,'Euro Indicator'] = df_all[df_all['year']>2019].groupby(['iso3'])['Euro Indicator'].transform('first')
test = lambda x: x/x.sum()
df_all['nanSumShares']=df_all['SumShares_I'].isnull()

# make adjustments based on the availability of each invoice share over time
df_all['notNanSS'] = df_all['SumShares'].notnull()
df_all['notNanSS_I'] = df_all['SumShares_I'].notnull()
df_all['notNanSS_X'] = df_all['SumShares_X'].notnull()
df_all['notNanSS_X_I'] = df_all['SumShares_X_I'].notnull()
df_all['notNanUSD'] = df_all['Import USD'].notnull()
df_all['notNanEUR'] = df_all['Import EUR'].notnull()


df_all.loc[:,'EU_GDP_SS']=df_all.groupby(['year','month','Euro Indicator','notNanSS'])['gdp_cur_dol'].transform(test)
df_all.loc[:,'EU_GDP_SS_I']=df_all.groupby(['year','month','Euro Indicator','notNanSS_I'])['gdp_cur_dol'].transform(test)
df_all.loc[:,'EU_GDP_SS_X']=df_all.groupby(['year','month','Euro Indicator','notNanSS_X'])['gdp_cur_dol'].transform(test)
df_all.loc[:,'EU_GDP_SS_X_I']=df_all.groupby(['year','month','Euro Indicator','notNanSS_X_I'])['gdp_cur_dol'].transform(test)
df_all.loc[:,'EU_GDP_USD']=df_all.groupby(['year','month','Euro Indicator','notNanUSD'])['gdp_cur_dol'].transform(test)
df_all.loc[:,'EU_GDP_EUR']=df_all.groupby(['year','month','Euro Indicator','notNanEUR'])['gdp_cur_dol'].transform(test)

# Now keep if in EU and calculate EU import shares
df_all['importShares_sum'] = df_all['SumShares']*df_all['EU_GDP_SS'] 
df_all['importShares_sum_I'] = df_all['SumShares_I']*df_all['EU_GDP_SS_I'] 
df_all['exportShares_sum'] = df_all['SumShares_X']*df_all['EU_GDP_SS_X'] 
df_all['exportShares_sum_I'] = df_all['SumShares_X_I']*df_all['EU_GDP_SS_X_I'] 
df_all['ImportUSD_sum'] = df_all['Import USD']*df_all['EU_GDP_USD'] 
df_all['ImportEUR_sum'] = df_all['Import EUR']*df_all['EU_GDP_EUR'] 


In [14]:
#### N.B. Estonia is not populated with EU_GDP because its data is not monthly!!!! I am ok with that!


df_all[(df_all['Euro Indicator']==True)&(df_all['gdp_cur_dol'].notnull())&(df_all['EU_GDP_SS'].isnull())]

,Country,year,iso3,imfcode,Import USD_I,Import EUR_I,Import Home_I,Import USD,Import EUR,Import Home,...,EU_GDP_SS_X,EU_GDP_SS_X_I,EU_GDP_USD,EU_GDP_EUR,importShares_sum,importShares_sum_I,exportShares_sum,exportShares_sum_I,ImportUSD_sum,ImportEUR_sum
16785,Estonia,2011,EST,939.0,5.201383,89.539376,0.0,5.201383,89.539376,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16786,Estonia,2012,EST,939.0,4.889512,89.633059,0.0,4.889512,89.633059,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16787,Estonia,2013,EST,939.0,4.074245,90.111267,0.0,4.074245,90.111267,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16788,Estonia,2014,EST,939.0,4.346998,90.215449,0.0,4.346998,90.215449,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16789,Estonia,2015,EST,939.0,3.974816,90.806253,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16790,Estonia,2016,EST,939.0,4.100208,90.239245,0.0,4.100208,90.239245,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16791,Estonia,2017,EST,939.0,3.859604,90.828514,0.0,3.859604,90.828514,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16792,Estonia,2018,EST,939.0,5.172734,89.092795,0.0,5.172734,89.092795,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16793,Estonia,2019,EST,939.0,4.898147,89.057148,0.0,4.898147,89.057148,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16794,Estonia,2020,EST,939.0,4.754295,89.378525,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
EUR = df_all[df_all['Euro Indicator']==True]

varslist = ['importShares_sum','importShares_sum_I','exportShares_sum','exportShares_sum_I','ImportUSD_sum','ImportEUR_sum','gdp_cur_dol']
EUR=EUR.groupby(['year','month'])[varslist].sum().reset_index()
EUR = EUR.rename(columns={'gdp_cur_dol':'EU_GDP_temp'})
df_all = df_all.drop(columns=varslist)

#varslist = ['SumShares','SumShares_I','SumShares_X','SumShares_X_I']
EUR['iso3']='EUR'

df_all = pd.merge(df_all,EUR,on=['iso3','year','month'],how='left')

df_all.loc[df_all['iso3']=='EUR','SumShares']=df_all.loc[df_all['iso3']=='EUR','importShares_sum']
df_all.loc[df_all['iso3']=='EUR','SumShares_I']=df_all.loc[df_all['iso3']=='EUR','importShares_sum_I']
df_all.loc[df_all['iso3']=='EUR','SumShares_X']=df_all.loc[df_all['iso3']=='EUR','exportShares_sum']
df_all.loc[df_all['iso3']=='EUR','SumShares_X_I']=df_all.loc[df_all['iso3']=='EUR','exportShares_sum_I']
df_all.loc[df_all['iso3']=='EUR','Import USD']=df_all.loc[df_all['iso3']=='EUR','ImportUSD_sum']
df_all.loc[df_all['iso3']=='EUR','Import EUR']=df_all.loc[df_all['iso3']=='EUR','ImportEUR_sum']
df_all.loc[(df_all['iso3']=='EUR')&(df_all['EU_GDP_temp']>0),'gdp']=df_all.loc[df_all['iso3']=='EUR','EU_GDP_temp']
varslist = ['importShares_sum','importShares_sum_I','exportShares_sum','exportShares_sum_I','EU_GDP_temp']
df_all = df_all.drop(columns = varslist)


#EUR=EUR.rename(columns={'importShares_sum':'SumShares','importShares_sum_I':'SumShares_I','exportShares_sum':'SumShares_X','exportShares_sum_I':'SumShares_X_I'})

#df_all.combine_first(EUR)
#df_all.loc[df_all['iso3']=='EUR',varslist] = EUR.loc[:,varslist]


# fix euro labelling
df_all.loc[df_all['iso3'] == 'EUR', 'ISO2'] = 'EU'
df_all.loc[df_all['iso3']=='EUR','Country']='EU'
df_all.loc[df_all['iso3'] == 'EUR', 'imfcode'] = 160

In [16]:
# Before saving remove all entries with months = nan (this means there was an outer merge with entries we can;t use)
df_all = df_all[df_all['month'].notnull()]

# drop uneeded variables
df_all = df_all.drop(columns=['EntryDate', 'nanSumShares',
       'notNanSS', 'notNanSS_I', 'notNanSS_X', 'notNanSS_X_I', 'EU_GDP_SS',
       'EU_GDP_SS_I', 'EU_GDP_SS_X', 'EU_GDP_SS_X_I'])

# save
df_all.to_pickle(path_cleandata+'FullData'+saveAppend+'.pkl')
df_all.to_csv(path_cleandata+'FullData'+saveAppend+'.csv')